<a href="https://colab.research.google.com/github/Biwako3320/AozoraTxt/blob/master/%E3%83%92%E3%82%99%E3%82%B7%E3%82%99%E3%83%8D%E3%82%B9%E3%82%A4%E3%83%B3%E3%83%86%E3%83%AA%E3%82%B7%E3%82%99%E3%82%A7%E3%83%B3%E3%82%B9_%E6%9C%80%E7%B5%82%E3%83%AC%E3%83%9B%E3%82%9A%E3%83%BC%E3%83%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 準備

In [2]:
# ライブラリは必要に応じて適宜追加すること
import numpy as np
import numpy.random as random
import scipy as sp
from pandas import Series,DataFrame
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline

import sklearn
from sklearn.decomposition import PCA

%precision 3

'%.3f'

# 問1
以下の用語について，それぞれどういうものか説明しなさい
- ビジネスインテリジェンス
- データサイエンス

# 問2
以下のURLからダウンロードできるデータは，ある金融機関のデータで，顧客が定期預金口座開設の申込をしたか否かの変数を含んでいる．このデータで顧客の分析を行う．

http://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip

これに含まれるbank-full.csvを分析対象とする．

データの項目の説明は以下の通り．データは”;”区切りになっている．

|項目名|説明|
|----|----|
|age|年齢(numeric)|
|job|職種(categorical: "admin.", "unknown”, "unemployed", ”management", "housemaid", "entrepreneur", "student", "blue-collar", "self-employed", "retired", "technician", "services”)|
|marital|婚姻状態(categorical: "married", "divorced", "single”)|
|education|教育(categorical: "unknown","secondary","primary","tertiary”)|
|default|債務不履行があったか(binary: “yes”, “no”)|
|balance|年間平均残高(ユーロ)(numeric)|
|housing|住宅ローンがあるか(binary: “yes”, “no”)|
|loan|個人ローンがあるか(binary: “yes”, “no”)|
|contact|連絡方法(categorical: "unknown","telephone","cellular”)|
|day|最終連絡日(numeric)|
|month|最終連絡月(categorical:”jan”,”feb”,”mar”,…,”nov”,”dec”)|
|duration|最終連絡時の長さ(numeric)|
|campaign|このキャンペーンで何度連絡を取ったか(numeric)|
|pdays|このキャンペーンを含め前の連絡からどのくらい経過したか(numeric, -1は連絡したことがない)|
|poutcome|このキャンペーンより前に何度連絡したか(numeric)|
|y|定額預金があるか(binary: “yes”, “no”)|


In [3]:
# データのダウンロード
%%bash
wget "http://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip"
unzip bank.zip

Archive:  bank.zip
  inflating: bank-full.csv           
  inflating: bank-names.txt          
  inflating: bank.csv                


--2024-01-15 14:07:40--  http://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘bank.zip’

     0K .......... .......... .......... .......... ..........  386K
    50K .......... .......... .......... .......... ..........  854K
   100K .......... .......... .......... .......... .......... 4.89M
   150K .......... .......... .......... .......... ..........  996K
   200K .......... .......... .......... .......... .......... 4.23M
   250K .......... .......... .......... .......... .......... 4.04M
   300K .......... .......... .......... .......... .......... 13.0M
   350K .......... .......... .......... .......... .......... 1.43M
   400K .......... .......... .......... .......... .......... 5.34M
   450K .......... .......... ...

In [4]:
# データの読み込み
bank=pd.read_csv("bank-full.csv",sep=";")
bank

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no


## 問2.1
数値データ(age,balance,day,duration,campaign,pdays,previous)における基本統計量(レコード数，最大値，最小値，標準偏差，分散，平均)を求めよ．

In [5]:
basic=bank[['age', 'balance','day','duration','campaign','pdays','previous']]
statistics = basic.describe()
print(statistics)

                age        balance           day      duration      campaign  \
count  45211.000000   45211.000000  45211.000000  45211.000000  45211.000000   
mean      40.936210    1362.272058     15.806419    258.163080      2.763841   
std       10.618762    3044.765829      8.322476    257.527812      3.098021   
min       18.000000   -8019.000000      1.000000      0.000000      1.000000   
25%       33.000000      72.000000      8.000000    103.000000      1.000000   
50%       39.000000     448.000000     16.000000    180.000000      2.000000   
75%       48.000000    1428.000000     21.000000    319.000000      3.000000   
max       95.000000  102127.000000     31.000000   4918.000000     63.000000   

              pdays      previous  
count  45211.000000  45211.000000  
mean      40.197828      0.580323  
std      100.128746      2.303441  
min       -1.000000      0.000000  
25%       -1.000000      0.000000  
50%       -1.000000      0.000000  
75%       -1.000000      0.

## 問2.2
データのjob,marital,education,default,housing,loanのそれぞれの値について，預金を申し込む人，申し込まない人の人数を求めよ．


In [6]:
saving=bank[['job', 'education','default','housing','loan','y']]
saving

,job,education,default,housing,loan,y
0,management,tertiary,no,yes,no,no
1,technician,secondary,no,yes,no,no
2,entrepreneur,secondary,no,yes,yes,no
3,blue-collar,unknown,no,yes,no,no
4,unknown,unknown,no,no,no,no
...,...,...,...,...,...,...
45206,technician,tertiary,no,no,no,yes
45207,retired,primary,no,no,no,yes
45208,retired,secondary,no,no,no,yes
45209,blue-collar,secondary,no,no,no,no


In [14]:
saving[(saving['job']=='management')&(saving['y']=='yes')]
#saving[(saving['job']=='management')&(saving['y']=='no')]
saving[(saving['job']=='admin')&(saving['y']=='yes')]
saving[(saving['job']=='unknown')&(saving['y']=='yes')]
saving[(saving['job']=='unemployed')&(saving['y']=='yes')]
saving[(saving['job']=='housemaid')&(saving['y']=='yes')]
saving[(saving['job']=='entrepreneur')&(saving['y']=='yes')]
saving[(saving['job']=='student')&(saving['y']=='yes')]
saving[(saving['job']=='blue-collar')&(saving['y']=='yes')]
saving[(saving['job']=='self-employed')&(saving['y']=='yes')]
saving[(saving['job']=='retired')&(saving['y']=='yes')]
saving[(saving['job']=='technician')&(saving['y']=='yes')]
saving[(saving['job']=='services')&(saving['y']=='yes')]

,job,education,default,housing,loan,y
129,services,secondary,no,yes,no,yes
569,services,secondary,no,yes,no,yes
891,services,primary,no,yes,yes,yes
1247,services,secondary,no,yes,no,yes
2387,services,secondary,yes,yes,no,yes
...,...,...,...,...,...,...
44903,services,secondary,no,no,no,yes
44913,services,secondary,no,no,no,yes
44951,services,unknown,no,no,no,yes
45185,services,tertiary,no,yes,no,yes


## 問2.3
y(預金を申し込む，申し込まない)を目的変数として，予測モデルを構築し，モデルの正解率(accuracy)とF値(f1-score)のマクロ平均(macro avg)を求めよ．
- 学習モデルとしてサポートベクトルマシン(sklearn.svm)を使用すること
- 説明変数は"age"，"balance","campaign"を選択すること
- 目的変数yについて，預金を申し込む("yes")を1，申し込まない("no")を0とすること

このデータは学習データ，テストデータに分割されていないので，sklearn.model_selection.train_test_splitを用いてデータ分割を行うこと．また，その際に，`stratify=<目的変数>, random_state=0`というオプションを指定する．


### 問2.4
問2.3の結果から，このモデルは実際の現場で運用できるかを考察せよ．

ヒント:
- 各ラベル毎の結果を見てみる
- 予測したラベルの分布を見てみる

# 問3
[台東区来訪者アンケート調査結果](https://www.city.taito.lg.jp/kusei/online/opendata/iryo/raihousha_anke-to.html)は台東区が2018年度に実施した来訪者の動向を把握するための調査の結果である．


## データの準備
台東区のサイトからデータをダウンロードして，文字コードを変換する．

In [7]:
%%bash
# nkfは文字コード変換用のコマンド
apt-get install nkf
wget "https://www.city.taito.lg.jp/kusei/online/opendata/iryo/raihousha_anke-to.files/03raihousha-kekka.csv"
# Shift-jisなのでutf-8に変換する
nkf -w 03raihousha-kekka.csv > 03raihousha-kekka.utf8.csv

wget "https://www.city.taito.lg.jp/kusei/online/opendata/iryo/raihousha_anke-to.files/02raihousha-code.csv"
# Shift-jisなのでutf-8に変換する
nkf -w 02raihousha-code.csv > 02raihousha-code.utf8.csv


Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  nkf
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 154 kB of archives.
After this operation, 367 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 nkf amd64 1:2.1.5-1build3 [154 kB]
Fetched 154 kB in 0s (557 kB/s)
Selecting previously unselected package nkf.
(Reading database ... 121654 files and directories currently installed.)
Preparing to unpack .../nkf_1%3a2.1.5-1build3_amd64.deb ...
Unpacking nkf (1:2.1.5-1build3) ...
Setting up nkf (1:2.1.5-1build3) ...
Processing triggers for man-db (2.10.2-1) ...


--2024-01-15 14:07:53--  https://www.city.taito.lg.jp/kusei/online/opendata/iryo/raihousha_anke-to.files/03raihousha-kekka.csv
Resolving www.city.taito.lg.jp (www.city.taito.lg.jp)... 202.247.51.71, 202.247.51.72, 111.108.184.60, ...
Connecting to www.city.taito.lg.jp (www.city.taito.lg.jp)|202.247.51.71|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 952099 (930K) [text/csv]
Saving to: ‘03raihousha-kekka.csv’

     0K .......... .......... .......... .......... ..........  5%  321K 3s
    50K .......... .......... .......... .......... .......... 10%  795K 2s
   100K .......... .......... .......... .......... .......... 16%  380K 2s
   150K .......... .......... .......... .......... .......... 21% 1011K 1s
   200K .......... .......... .......... .......... .......... 26%  820K 1s
   250K .......... .......... .......... .......... .......... 32% 1.22M 1s
   300K .......... .......... .......... .......... .......... 37% 1.35M 1s
   350K .......... ........

In [8]:
# 回答結果の読み込み
questionnaire=pd.read_csv("03raihousha-kekka.utf8.csv")

In [9]:
# 確認
questionnaire.head()

,管理番号,調査時期,調査地点ID,調査年月日,調査時刻,問１_居住地区分,問１_都道府県名,問１_区市町村名,問１_国名,問１_入国空港・港,...,問⑭_残念-1,問⑭_1-理由.1,問⑭_残念-2,問⑭_2-理由.1,問⑮_大きな荷物の有無,問⑮_大きな荷物理由,問⑮_4.その他/理由,問⑮_預かり所等の活用,問⑮_料金,問⑯_意見
0,1,1,3,2018/6/30,10:00,2,富山県,富山市,NaN,NaN,...,14,暑さが厳しい,999,NaN,2,999,NaN,2,NaN,街の景観を保つために努力してほしい（ゴミ、トイレetc）
1,2,1,3,2018/6/30,10:04,2,愛知県,半田市,NaN,NaN,...,6,混雑が激しい,999,NaN,2,4,バスに預けている,999,NaN,今度はプライベートで来たい
2,3,1,3,2018/6/30,10:11,1,東京都,江戸川区,NaN,NaN,...,6,人が多すぎる,999,NaN,2,999,NaN,2,NaN,NaN
3,4,1,3,2018/6/30,10:24,2,静岡県,焼津市,NaN,NaN,...,6,人の多さが嫌,999,NaN,2,4,ホテルにあずけてある,2,NaN,駅からの案内を分かりやすくして、他言語に対する対応を強化
4,5,1,3,2018/6/30,11:11,2,島根県,益田市,NaN,NaN,...,4,トイレが少ないし、わかりにくい,14,自動の歩道があるといいと思う,2,4,コインロッカーに預けた,1,500.0,道路の整備を強化してほしい


## 問3.1
この回答結果から，台東区の来訪者の特徴を答えよ(例えば，どこから来た人が多いのか，何を目的にしている人が多いのか，等)．

## 問3.2
"問⑯_意見"の回答から，より多くの来訪者が訪れ，満足してもらうために台東区として取り組むべきこととして考えられることを答えよ．